In [ ]:
import datetime
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
%matplotlib inline
#%matplotlib tk
'''
import logging

logging.basicConfig(filename='c14.log',
                              filemode='a',
                              format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                             datefmt='%H:%M:%S',
                             level=logging.DEBUG)
'''                 
pd.set_option('display.max_rows', 500)

In [ ]:
import arviz as az
from matplotlib import rcParams,rc
import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
fonts = 1
sb.set(context="paper",style='ticks',font_scale=fonts)
plt.rcParams["font.family"] = "Times New Roman"

In [ ]:
%load_ext autoreload
%autoreload 2
import c14
import c14.models.liver_new_mitosis as lm
import c14.corner
from c14.utils import *


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
directory = '/raid/uni/data/jrode-heart/liverF_IS_newploidy/'
data = pd.read_csv('C14data_liver_samples_20211020.csv')
data = data.groupby(['type', 'sample', 'ploidy', 'pathology']).mean().dropna(how='all').reset_index()
data['age'] = data['Dcoll'] - data['Dbirth']
data = data.query('type == "hepatocyte" and pathology != ["Y", "C"]')
data['measurment_types'] = data['ploidy']





In [ ]:
len(data)

In [ ]:
np.seterr(all='ignore')
results= dict()
NUM=0
results= dict(data=data,results={},models={})


for mm in lm.models_list :
    m = mm(dnatotal=lm.dnatotal_spline)
    if m.__class__.__name__ not in ['POP1','Rm','POP3p_2x2n_NP']:
        continue
        pass
    path = directory+m.__class__.__name__+'spline_'+str(NUM) + '.pickle'
    try:
        with open(path, 'rb') as handle:
            res = pickle.load(handle)
    except FileNotFoundError:
        print(m.__class__.__name__,path)
        continue
    except:
        print(path)
        continue
    #print('FOUND',mm.__name__)
    results['results'].update(  {m.__class__.__name__+'spline' : {'raw':res} } )
    results['models'].update(   {m.__class__.__name__+'spline' : m } )        


get_arviz(results,burnin=1000)
ranking = get_getranking(results)
ranking
printt = get_pointestimates(results,error_in_real=False)[['Parameter','Value','Confidence Interval']]
printt.to_excel('liver_fix_is_newploidy.xlsx')
calc_c14_all(results)
printt

In [ ]:
from scipy.stats import normaltest
from scipy.stats import shapiro

In [ ]:
delta_val = {i :[] for i in results['models'].keys()}
for i,(model_name,model) in enumerate(results['models'].items()):
    results_m = results['results'][model_name]
    for subject,row in data.iterrows():
        sigf = results['point_est'].loc[model_name,('median','sigma')]
        delta = (results_m['c14_M'].loc[subject]-row['d14C'])/np.sqrt(row['e14C']**2+sigf**2)
        if np.abs(delta)<0.1 or True:
            delta_val[model_name].append(delta)
    print(model_name,normaltest(delta_val[model_name]))
    print(model_name,shapiro(delta_val[model_name]))
    
    plt.hist(delta_val[model_name],density=True)
    plt.title(model_name)
    plt.figure()

In [ ]:
from collections import defaultdict


subject = 1
excel_writer = pd.ExcelWriter('time_rates_flows_newploidy.xlsx')
for m_i,(model_name,model) in enumerate(results['models'].items()):
    print(model_name)
    age = results['data'].loc[subject,'age']
    T = np.arange(0,90,0.1)
    parameter_phy =  {i:results['point_est'].loc[model_name].loc[('median',i)] for i in model.parameter_names}
    model.set_parameters_phy(parameter_phy,mode='bayes')
    ipsd=[]
    for i in T:
        ip = model.calc_implicit_parameters(i)
        ipsd.append(ip)

    a_i =  [j for j,x in enumerate(results['data'].index) if x == subject][0]  
    ipid = listofdict_to_dictofarray_f(ipsd,a_i)
    ipid.update({n:np.array([v]*len(T)) for n,v in parameter_phy.items()})
    df = pd.DataFrame(ipid)
    df.index.name = 'age'
    df.index =df.index/10
    
    
    flow_name = defaultdict(list)
    for i,(ind2,flows) in enumerate(model.flow_in.items()):
        y = np.zeros_like(T)
        for rate,pop,factor in flows:
            try:
                if pop is None:
                    y += ipid[rate] * factor
                else:
                    y += ipid[rate] * factor * ipid[pop]
            except Exception as e:
                raise Exception([e,Exception(f'in class {model_name}')])
        #df['inflow_'+ind2] = y
        for rate,pop,factor in flows:
            try:
                if pop is None:
                    yi = ipid[rate] * factor
                else:
                    yi = ipid[rate] * factor * ipid[pop]
            except Exception as e:
                raise Exception([e,Exception(f'in class {model_name}')])
            df['probIN_'+pop+'_to_'+ind2] = yi/y
            df['flow_'+pop+'_to_'+ind2] = yi
            flow_name[pop] += [ind2]
        #print(ind2,flows)
    for pop_source,val in flow_name.items():
        norm = 0
        for pop in val:
            norm += df['flow_'+pop_source+'_to_'+pop]
        for pop in val:
            df['probOUT_'+pop_source+'_to_'+pop] = df['flow_'+pop_source+'_to_'+pop]/norm
    df.to_excel(excel_writer=excel_writer,sheet_name=model_name)
    if model_name =='Rmspline':
        print(df[['b2n','b4n']].loc[50]*np.array([model.ploidy(50),1-model.ploidy(50)])*1e11*1/365/1e6)
        N50 = np.sum(df[['N4n','N2n']].loc[50])
        print('N2',np.sum(df[['flow_N2n_to_N2n','flow_N4n_to_N2n']].loc[50])/N50*1e11*1/365/1e6)
        print('N4',np.sum(df[['flow_N2n_to_N4n','flow_N4n_to_N4n']].loc[50])/N50*1e11*1/365/1e6)
    if model_name =='POP3p_2x2n_NPspline':
        print(df[['g2nb2n','k2x2nb2x2n','k4nb4n']].loc[50]*np.array([model.ploidy(50),model.ploidy2x2(50),1-model.ploidy2x2(50)-model.ploidy(50)])*1e11*1/365/1e6)
        N50 = np.sum(df[['N4n','N2x2n','N2n']].loc[50])
        NN50 = 2.5225*1e11
        print('N2',np.sum(df[['flow_N2x2n_to_N2n','flow_N2n_to_N2n','flow_N4n_to_N2n']].loc[50])/N50*NN50*1/365/1e6)
        print('N2x2',np.sum(df[['flow_N2x2n_to_N2x2n','flow_N4n_to_N2x2n']].loc[50])/N50*NN50*1/365/1e6)
        print('N4',np.sum(df[['flow_N2x2n_to_N4n','flow_N4n_to_N4n']].loc[50])/N50*NN50*1/365/1e6)

excel_writer.save()

In [ ]:
with pd.HDFStore('point_est.pandas') as st:
    st['F_IS_spline_newploidy'] = results['point_est']



In [ ]:
rename={'POP3p_2x2n_NPspline':{'sigma':r'$\sigma$','d2n':r'$\delta_2$','d2x2n':r'$\delta_{2x2}$','d4n':r'$\detla_4$',
                               'k2nb4n':r'$\kappa_{4,2}$','g2x2nb2n':r'$\kappa_{2x2,2}$','k2nb2x2n':r'$\kappa_{2,2x2}$'},
        'Rmspline':{'sigma':r'$\sigma$','d2n':r'$\delta_2$','d4n':r"$\delta_{p}$",'k24':r"$\kappa_{2p}$",'k42':r"$\kappa_{p2}$"},
        'POP1spline':{'sigma':r'$\sigma$','d':r'$\delta$'}}


In [ ]:
import os
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.patches as patches
import matplotlib as mpl
for n,v in results['results'].items():  
    fimag = 'overviewF_IS_'+n+'.png'
    if not os.path.isfile(fimag) or True :
        c14.corner.corner_R(v['azdata'].posterior.a.values,
                   results['models'][n].parameter_names,
                        point_estimate=results['models'][n].transform_physical_to_fit(v['median'],mode='bayes'),
                        rename=rename[n],unitlog='[1/year]',
                        logparas=results['models'][n].logparas)    
        f = plt.gcf()
        f.show();
        if n=='POP1spline':
            bbax0 = f.get_axes()[0].get_position()
            bbax3 = f.get_axes()[2].get_position()
            ticks = [-1,0]
            f.get_axes()[0].set_xticks(ticks)
            f.get_axes()[0].set_xticklabels([r"$10^{"+str(int(i))+r"}$"  for i in ticks])
            
            f.get_axes()[1].set_xticks(ticks)
            f.get_axes()[1].set_xticklabels([r"$10^{"+str(int(i))+r"}$"  for i in ticks])
            
            axL=plt.axes([bbax3.x0, bbax0.y0+bbax0.height*0.1,  bbax3.width, bbax0.height*.5], frameon=False, xticks=[],yticks=[])
            axL.set_xlim((0,1))
            axL.set_ylim((0,1))
            ellipse = patches.Ellipse(xy=(0.1,0.8), height=0.2, width=0.15, angle=0 ,      edgecolor='red', fc='None', lw=1)
            axL.text(0.35, 0.75, "1-sigma CR")
            axL.add_patch(ellipse)
            axC=plt.axes([bbax3.x0, bbax0.y0+bbax0.height*0.1,bbax3.width,bbax0.height*0.1], frameon=True)
            norm = mpl.colors.Normalize(vmin=0,vmax=1)
            sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis, norm=norm)
            sm.set_array([])
            cbar = plt.colorbar(sm,cax=axC,orientation='horizontal')
            cbar.set_ticks([0,1])
            cbar.set_ticklabels(['low','high'])
            axC.set_title('Probability density')
            axC.xaxis.get_major_ticks()[0].label1.set_horizontalalignment('left')
            axC.xaxis.get_major_ticks()[1].label1.set_horizontalalignment('right')
            #f.suptitle('hepato ' + n,fontsize=30)
            plt.tight_layout()    
        f.savefig(fimag,dpi=600)
